# Load VGG16 Model

In [2]:
from keras.applications import vgg16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Loads the VGG16 model without the top or FC layers
model = vgg16.VGG16(weights='imagenet', include_top = False,
                    input_shape=(img_rows,img_cols,3))


### Inspecting each layer

In [3]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


### Let's freeze all layers except the top 4 

In [4]:
# Here we freeze the layers 
# Layers are set to trainable as True by default

for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


### Let's make a function that returns our FC Head (fully connected)

In [5]:
def addTopModel(bottom_model, num_classes, D=256):
    # creates the top or head of the model that will be placed ontop of the bottom layers

    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [ ]:
model.input

In [ ]:
model.layers

### Let's add our FC Head back onto VGG

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set the number of classes
num_classes = 2  #since two people's face to be recognised

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs = model.input, outputs = FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Loading our Created Dataset

In [8]:
#image augmentation

from keras_preprocessing.image import ImageDataGenerator

train_data_dir = 'Dataset/train/'
validation_data_dir = 'Dataset/test/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Set the Batch Size according to your system.
train_batchsize = 16
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


### Training Model

In [9]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("facedetect.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# We put our call backs into a callback list
callbacks = [earlystop, checkpoint]

modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 200     #training
nb_validation_samples = 100     #testing/validation

# We will be training our model for 5 EPOCHS 
epochs = 5
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)



Epoch 1/5
12/12 [==============================] - 146s 12s/step - loss: 4.5307 - accuracy: 0.5761 - val_loss: 0.4665 - val_accuracy: 0.9167
Epoch 2/5
12/12 [==============================] - 144s 12s/step - loss: 0.9814 - accuracy: 0.7174 - val_loss: 0.0018 - val_accuracy: 0.7500
Epoch 3/5
12/12 [==============================] - 147s 12s/step - loss: 0.3728 - accuracy: 0.8261 - val_loss: 0.0824 - val_accuracy: 0.9833
Epoch 4/5
12/12 [==============================] - 142s 12s/step - loss: 0.1168 - accuracy: 0.9674 - val_loss: 0.0096 - val_accuracy: 0.9500
Epoch 5/5
12/12 [==============================] - 147s 12s/step - loss: 0.3674 - accuracy: 0.8587 - val_loss: 0.0140 - val_accuracy: 0.9333


In [ ]:
#To save this trained model:
#modelnew.save("facedetect.h5")

In [10]:
all_faces= train_generator.class_indices
print(all_faces)

{'ansh': 0, 'raj': 1}


{'ansh': 0, 'raj': 1}

### Testing

In [18]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

model = load_model("facedetect.h5")

#Path to my sample pic
filename = 'Dataset/sample/face5' + '.jpg'

test_img = image.load_img(filename, target_size=(224,224,3))

test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img, axis=0)
result = model.predict(test_img)

print(result)

if result[0][0]==1:
    print("its you ansh")
elif result[0][1]==1:
    print("its you raj")
else:
    print("Face Not Match")

[[1.0000000e+00 1.1981449e-23]]
its you ansh
